# Generative AI Use Case: Summarize Dialogue

In [6]:
%pip install datasets
%pip install transformers
%pip install torch
%pip install torchdata


                                              0.0/486.2 kB ? eta -:--:--
                                              0.0/486.2 kB ? eta -:--:--
                                              0.0/486.2 kB ? eta -:--:--
     --                                       30.7/486.2 kB ? eta -:--:--
     ---                                   41.0/486.2 kB 667.8 kB/s eta 0:00:01
     ---                                   41.0/486.2 kB 667.8 kB/s eta 0:00:01
     -----                                 71.7/486.2 kB 491.5 kB/s eta 0:00:01
     -------                               92.2/486.2 kB 438.1 kB/s eta 0:00:01
     -------                               92.2/486.2 kB 438.1 kB/s eta 0:00:01
     ---------                            122.9/486.2 kB 400.9 kB/s eta 0:00:01
     -----------                          153.6/486.2 kB 459.5 kB/s eta 0:00:01
     -----------                          153.6/486.2 kB 459.5 kB/s eta 0:00:01
     -----------                          153.6/486.2 kB 459.5 kB/s

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GenerationConfig

In [8]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 71.56it/s]


Dataset csv downloaded and prepared to C:/Users/astaj/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c07c4cf4362c223c/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 41.78it/s]


In [9]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print("Example {}".format(i+1))
    print(dash_line)
    print('Input Dialogue:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('baseline summary:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()
    

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
baseline summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example 2


In [10]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

C:\Users\astaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\astaj\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [12]:
sentence = "Hello, my dog is cute"

sentence_encoded = tokenizer(sentence, return_tensors="pt")

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0], skip_special_tokens=True)

print("Encoded sentence: {}".format(sentence_encoded))
print("Decoded sentence: {}".format(sentence_decoded))


Encoded sentence: {'input_ids': tensor([[8774,    6,   82, 1782,   19, 5295,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Decoded sentence: Hello, my dog is cute


### Without any Prompt Engineering

In [ ]:
HI